# Final Emissions ETL

In [56]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-upda

In [57]:
# Import SparkSession and great expectations 
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
# Start Spark session
from pyspark import SparkFiles

In [58]:
import time
start_time = time.time()

### Pytest for Emissions and Temp Datasets

In [88]:
%%file midterm_data.py
# Import findspark() and initialize. 
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkBankData").getOrCreate()

def data_emissions():
  emissions = spark.read.csv('oecd_data_cleaned.csv', header=True)
  return emissions

def data_temp():
  temp_by_yr = spark.read.csv('Temp_by_Year.csv', header=True)
  return temp_by_yr


1368


In [95]:
%%file testing_midterm_data.py
from midterm_data import data_emissions
from midterm_data import data_temp

def test_data_emissions():
  df = data_emissions()
  assert df.count() > 1368 #Includes other types of greenhouse gasses

def test_data_temp():
  df = data_temp()
  assert df.count() == 1368 #57 countries * 24 years = 1368 rows of data


Overwriting testing_midterm_data.py


In [96]:
!python -m pytest testing_midterm_data.py

============================= test session starts ==============================
platform linux -- Python 3.7.14, pytest-3.6.4, py-1.11.0, pluggy-0.7.1
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 2 items                                                              

testing_midterm_data.py ..                                               [100%]

========================== 2 passed in 17.47 seconds ===========================


### SQL Queries on 4 Questions Using Views

In [59]:
emissions_df = spark.read.csv('oecd_data_cleaned.csv', header=True)


In [60]:
emissions_df.show()

+---------+----------------+--------------------+------------+--------------------+---------+-----------+
|  Country|       Pollutant|            Variable|Emissionyear|                Unit|PowerCode|Emissionval|
+---------+----------------+--------------------+------------+--------------------+---------+-----------+
|Australia|Greenhouse gases|Total  emissions ...|        1990|Tonnes of CO2 equ...|Thousands| 425624.307|
|Australia|Greenhouse gases|Total  emissions ...|        1991|Tonnes of CO2 equ...|Thousands| 425686.445|
|Australia|Greenhouse gases|Total  emissions ...|        1992|Tonnes of CO2 equ...|Thousands| 429473.085|
|Australia|Greenhouse gases|Total  emissions ...|        1993|Tonnes of CO2 equ...|Thousands| 430381.731|
|Australia|Greenhouse gases|Total  emissions ...|        1994|Tonnes of CO2 equ...|Thousands| 430848.498|
|Australia|Greenhouse gases|Total  emissions ...|        1995|Tonnes of CO2 equ...|Thousands| 439269.878|
|Australia|Greenhouse gases|Total  emissions .

In [61]:
temperatures_df = spark.read.csv('temperatures_data_cleaned.csv', header=True)


In [62]:
temperatures_df.show()

+------------------+-----------------------------+---------+--------+---------+-------+
|AverageTemperature|AverageTemperatureUncertainty|  Country|Tempyear|Tempmonth|Tempday|
+------------------+-----------------------------+---------+--------+---------+-------+
|            22.032|                        0.189|Argentina|    1990|        1|      1|
|            20.844|                         0.19|Argentina|    1990|        2|      1|
|            18.194|                         0.24|Argentina|    1990|        3|      1|
|            14.242|                        0.228|Argentina|    1990|        4|      1|
|            10.831|                        0.226|Argentina|    1990|        5|      1|
|             8.023|                        0.331|Argentina|    1990|        6|      1|
|              7.88|                        0.154|Argentina|    1990|        7|      1|
|            11.754|                        0.245|Argentina|    1990|        8|      1|
|            11.703|            

In [63]:
temp_by_yr = spark.read.csv('Temp_by_Year.csv', header=True)

In [64]:
temp_by_yr.show()

+---------+-----+------------------+-------------------+
|  country|year_|   yearly_avg_temp|  yearly_avg_uncert|
+---------+-----+------------------+-------------------+
|Argentina| 1990|15.155083333333332|0.23374999999999999|
|Argentina| 1991|15.097583333333338|              0.169|
|Argentina| 1992|14.739166666666668|0.26233333333333336|
|Argentina| 1993|15.098666666666668|0.20533333333333334|
|Argentina| 1994|15.578666666666669|0.22458333333333333|
|Argentina| 1995|15.149833333333333|0.22383333333333333|
|Argentina| 1996|15.211583333333332|0.23149999999999996|
|Argentina| 1997|15.620666666666667|0.20916666666666664|
|Argentina| 1998|15.261750000000001|0.20041666666666666|
|Argentina| 1999|15.064916666666667|0.19266666666666662|
|Argentina| 2000|           14.7505|0.19799999999999998|
|Argentina| 2001|15.370416666666666|0.21049999999999994|
|Argentina| 2002|15.183750000000003|            0.19925|
|Argentina| 2003|15.518333333333336| 0.2540833333333334|
|Argentina| 2004|15.44216666666

In [65]:
# QUESTION 1 PART A
temp_by_yr.createOrReplaceTempView('temp_by_yr')

question_1a = spark.sql("""select country, yearly_avg_temp, year_
from temp_by_yr
where year_=1990
group by country, year_, yearly_avg_temp""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+------------------+-----+
|             country|   yearly_avg_temp|year_|
+--------------------+------------------+-----+
|             Estonia| 6.698583333333332| 1990|
|         New Zealand| 11.26933333333333| 1990|
|            Portugal|15.742750000000001| 1990|
|             Iceland|           1.77475| 1990|
|           Lithuania| 7.978416666666667| 1990|
|              Norway|            2.0665| 1990|
|               Italy|13.716916666666664| 1990|
|             Romania|            10.226| 1990|
|         Switzerland| 8.255583333333334| 1990|
|             Hungary|10.920333333333334| 1990|
|              Greece|15.499333333333333| 1990|
|          Costa Rica|26.290249999999997| 1990|
|              Cyprus| 19.19441666666667| 1990|
|             Ukraine| 9.400833333333333| 1990|
|              Russia|-4.022916666666667| 1990|
|      Czech Republic| 8.962666666666665| 1990|
|             Austria|             7.344| 1990|
|              Brazil| 25.27491666666667

In [66]:
# QUESTION 1 PART B
question_1b = spark.sql("""
select country, yearly_avg_temp, year_
from temp_by_yr
where year_=2013
group by country, year_, yearly_avg_temp;""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+-------------------+-----+
|             country|    yearly_avg_temp|year_|
+--------------------+-------------------+-----+
|         Netherlands|             9.8565| 2013|
|          Kazakhstan|            8.93725| 2013|
|              Latvia|           7.322375| 2013|
|             Ireland|            9.97475| 2013|
|              Greece|            17.1535| 2013|
|             Austria|           8.055125| 2013|
|           Indonesia|             26.467| 2013|
|             Finland|           4.064125| 2013|
|              Monaco|             10.417| 2013|
|             Iceland| 2.9202500000000002| 2013|
|        Saudi Arabia|           27.74075| 2013|
|       United States| 11.297111111111112| 2013|
|             Denmark|-14.927249999999999| 2013|
|             Germany|            9.23525| 2013|
|               Chile|  9.877374999999997| 2013|
|               Italy|             14.165| 2013|
|          Costa Rica| 26.457555555555555| 2013|
|China (People's R..

In [67]:
# QUESTION 2 PART A
emissions_df.createOrReplaceTempView('emissions')

question_2a = spark.sql("""select country, 
pollutant, 
powercode, 
variable,
Emissionval, 
Unit, 
Emissionyear
from emissions
where pollutant='Greenhouse gases' and Emissionyear = 1990 and variable= 'Total  emissions excluding LULUCF'""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------+----------------+---------+--------------------+-----------+--------------------+------------+
|       country|       pollutant|powercode|            variable|Emissionval|                Unit|Emissionyear|
+--------------+----------------+---------+--------------------+-----------+--------------------+------------+
|     Australia|Greenhouse gases|Thousands|Total  emissions ...| 425624.307|Tonnes of CO2 equ...|        1990|
|       Austria|Greenhouse gases|Thousands|Total  emissions ...|  78423.221|Tonnes of CO2 equ...|        1990|
|       Belgium|Greenhouse gases|Thousands|Total  emissions ...| 145686.757|Tonnes of CO2 equ...|        1990|
|        Canada|Greenhouse gases|Thousands|Total  emissions ...| 594722.243|Tonnes of CO2 equ...|        1990|
|Czech Republic|Greenhouse gases|Thousands|Total  emissions ...| 196955.242|Tonnes of CO2 equ...|        1990|
|       Denmark|Greenhouse gases|Thousands|Total  emissions ...|  71429.972|Tonnes of CO2 equ...|        1990|
|

In [68]:
# QUESTION 2 PART B
question_2b = spark.sql("""select country, 
pollutant, 
powercode, 
variable,
Emissionval, 
Unit, 
Emissionyear
from emissions
where pollutant='Greenhouse gases' and Emissionyear = 2013 and variable= 'Total  emissions excluding LULUCF'""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------+----------------+---------+--------------------+-----------+--------------------+------------+
|       country|       pollutant|powercode|            variable|Emissionval|                Unit|Emissionyear|
+--------------+----------------+---------+--------------------+-----------+--------------------+------------+
|     Australia|Greenhouse gases|Thousands|Total  emissions ...| 532267.335|Tonnes of CO2 equ...|        2013|
|       Austria|Greenhouse gases|Thousands|Total  emissions ...|  79772.019|Tonnes of CO2 equ...|        2013|
|       Belgium|Greenhouse gases|Thousands|Total  emissions ...|  120458.32|Tonnes of CO2 equ...|        2013|
|        Canada|Greenhouse gases|Thousands|Total  emissions ...| 732162.302|Tonnes of CO2 equ...|        2013|
|Czech Republic|Greenhouse gases|Thousands|Total  emissions ...| 128913.027|Tonnes of CO2 equ...|        2013|
|       Denmark|Greenhouse gases|Thousands|Total  emissions ...|   56919.35|Tonnes of CO2 equ...|        2013|
|

In [69]:
# QUESTION 3

t_1990 =  spark.read.csv('temp_1990.csv', header=True)
t_2013 = spark.read.csv('temp_2013.csv', header=True)

t_1990.createOrReplaceTempView('temp_1990')
t_2013.createOrReplaceTempView('temp_2013')

question_3 = spark.sql("""select temp_1990.country, (temp_2013.yearly_avg_temp - temp_1990.yearly_avg_temp) as difference_in_temp
from temp_2013
join temp_1990 on temp_1990.country=temp_2013.country
order by difference_in_temp desc;""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+------------------+
|             country|difference_in_temp|
+--------------------+------------------+
|              Canada|3.3812500000000005|
|             Denmark| 2.943083333333332|
|          Kazakhstan|            2.3815|
|                Iran| 2.114583333333332|
|            TÃ¼rkiye|1.9941250000000004|
|            Bulgaria|1.8142083333333314|
|       United States|1.7755277777777785|
|              Russia|1.7597916666666658|
|        Saudi Arabia|1.7219999999999906|
|China (People's R...|1.6590000000000016|
|              Greece|1.6541666666666686|
|             Ukraine|1.5126666666666662|
|             Romania|1.4194999999999993|
|             Hungary| 1.381166666666667|
|              Mexico| 1.258388888888895|
|               India|1.2350833333333355|
|              Cyprus| 1.155458333333332|
|             Croatia|1.1545833333333366|
|             Iceland|1.1455000000000002|
|             Finland| 1.120874999999999|
+--------------------+------------

In [70]:
# QUESTION 4

e_1990 =  spark.read.csv('Emission_1990.csv', header=True)
e_2013 = spark.read.csv('Emission_2013.csv', header=True)


e_1990.createOrReplaceTempView('emission_1990')
e_2013.createOrReplaceTempView('emission_2013')

question_4 = spark.sql("""select emission_1990.country, (emission_2013.value_ - emission_1990.value_) as difference_in_emission
from emission_2013
join emission_1990 on emission_1990.country=emission_2013.country
order by difference_in_emission desc;""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------------+----------------------+
|          country|difference_in_emission|
+-----------------+----------------------+
|     OECD America|              819633.0|
|     OECD - Total|              778520.0|
|OECD Asia Oceania|              683937.0|
|        Indonesia|             529760.61|
|           Brazil|     478344.8500000001|
|            Korea|    405210.98000000004|
|    United States|    331043.99399999995|
|           Mexico|            268040.065|
|         TÃ¼rkiye|    220328.69300000003|
|           Canada|            137440.059|
|            Japan|    136910.14000000013|
|        Australia|    106643.02799999999|
|        Argentina|    100771.12000000002|
|         Colombia|    60776.990000000005|
|            Chile|              53468.89|
|            Spain|              33727.19|
|      New Zealand|    14608.673999999999|
|       Costa Rica|               5539.18|
|         Portugal|              5082.178|
|          Ireland|     4154.881000000001|
+----------

In [71]:
## UnCaching and Caching Question 4

In [76]:
spark.sql("uncache table Emission_1990")

question_4 = spark.sql("""select emission_1990.country, (emission_2013.value_ - emission_1990.value_) as difference_in_emission
from emission_2013
join emission_1990 on emission_1990.country=emission_2013.country
order by difference_in_emission desc;""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------------+----------------------+
|          country|difference_in_emission|
+-----------------+----------------------+
|     OECD America|              819633.0|
|     OECD - Total|              778520.0|
|OECD Asia Oceania|              683937.0|
|        Indonesia|             529760.61|
|           Brazil|     478344.8500000001|
|            Korea|    405210.98000000004|
|    United States|    331043.99399999995|
|           Mexico|            268040.065|
|         TÃ¼rkiye|    220328.69300000003|
|           Canada|            137440.059|
|            Japan|    136910.14000000013|
|        Australia|    106643.02799999999|
|        Argentina|    100771.12000000002|
|         Colombia|    60776.990000000005|
|            Chile|              53468.89|
|            Spain|              33727.19|
|      New Zealand|    14608.673999999999|
|       Costa Rica|               5539.18|
|         Portugal|              5082.178|
|          Ireland|     4154.881000000001|
+----------

In [77]:
spark.sql("cache table Emission_1990")

question_4 = spark.sql("""select emission_1990.country, (emission_2013.value_ - emission_1990.value_) as difference_in_emission
from emission_2013
join emission_1990 on emission_1990.country=emission_2013.country
order by difference_in_emission desc;""").show()

print("--- %s seconds ---" % (time.time() - start_time))
#Note: Need to fix as this is running slower than just using a view.

+-----------------+----------------------+
|          country|difference_in_emission|
+-----------------+----------------------+
|     OECD America|              819633.0|
|     OECD - Total|              778520.0|
|OECD Asia Oceania|              683937.0|
|        Indonesia|             529760.61|
|           Brazil|     478344.8500000001|
|            Korea|    405210.98000000004|
|    United States|    331043.99399999995|
|           Mexico|            268040.065|
|         TÃ¼rkiye|    220328.69300000003|
|           Canada|            137440.059|
|            Japan|    136910.14000000013|
|        Australia|    106643.02799999999|
|        Argentina|    100771.12000000002|
|         Colombia|    60776.990000000005|
|            Chile|              53468.89|
|            Spain|              33727.19|
|      New Zealand|    14608.673999999999|
|       Costa Rica|               5539.18|
|         Portugal|              5082.178|
|          Ireland|     4154.881000000001|
+----------

In [ ]:
### Parquet for Question #4:

In [ ]:
## 1990 Emissions
# e_1990.write.parquet('emission_1990', mode='overwrite')
# e90_df=spark.read.parquet('emission_1990')


In [ ]:
## 2013 Emissions
# e_2013.write.parquet('emission_2013', mode='overwrite')
# e13_df=spark.read.parquet('emission_2013')

In [ ]:
# e90_df.createOrReplaceTempView('1990_emissions')
# e13_df.createOrReplaceTempView('2013_emissions')

In [ ]:
# question_4_par = spark.sql("""select e1990.country, (e2013.value_ - e1990.value_) as difference_in_emission
# from 2013_emissions as e2013
# join 1990_emissions as e1990 on e1990.country=e2013.country
# order by difference_in_emission desc;""").show()

# print("--- %s seconds ---" % (time.time() - start_time))